In [1]:
# HellaSwag

In [2]:
!pip install datasets
!pip install openpyxl
!pip install -q -U google-genai
# !pip install transformers
# !pip install accelerate
# !pip install peft
# !pip install bitsandbytes
# Imports
%load_ext autoreload
%autoreload 2

import pandas as pd
import sys

# Call models
from src.call_models import bedrock_connect, call_claude_bedrock
from src.call_models import google_connect, call_gemini, all_string_gemini_config, all_int_gemini_config
from src.translate_func import claude_translation, gemini_translation

# Datasets

from src.benchmarks_code import hellaswag
from prompts import hallaswag_prompts

# Access keys
from my_access_keys import google_access_key, aws_access_key, aws_secret_key

# .csv utils
from src.save_utils import add_dataset_to_csv

# Remove annoying warning
from IPython.core.display_functions import display

In [3]:
## GetDataset

In [4]:
bedrock_client = bedrock_connect(aws_access_key, aws_secret_key)
google_client = google_connect(google_access_key)

In [5]:
hellaswag_dataset = hellaswag.get_hellaswag_datasets()
hellaswag_dataset['hellaswag_train'] = hellaswag_dataset['hellaswag_train'].take(50)
hellaswag_file_name = 'compare_csv/hellaswag_train_top_50.csv'

print(hellaswag_file_name)
hellaswag_dataset

compare_csv/hellaswag_train_top_50.csv


{'hellaswag_train': Dataset({
     features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label'],
     num_rows: 50
 })}

In [6]:
df = add_dataset_to_csv(hellaswag_file_name, 'original', hellaswag_dataset['hellaswag_train'], hellaswag.hellaswag_sample_to_dict)
text_df = add_dataset_to_csv(hellaswag_file_name[:-4] + '-text.csv', 'original', hellaswag_dataset['hellaswag_train'], hellaswag.hellaswag_sample_to_dict)
display(df.head(2))
display(text_df.head(2))

,original,claude_v1_refine
0,<activity_label>Removing ice from car</activit...,<activity_label>הסרת קרח מהרכב</activity_label...
1,<activity_label>Baking cookies</activity_label...,<activity_label>אפיית עוגיות</activity_label>\...


,original
0,<activity_label>Removing ice from car</activit...
1,<activity_label>Baking cookies</activity_label...


In [7]:
df = pd.read_csv(hellaswag_file_name)
text_df = pd.read_csv(hellaswag_file_name[:-4] + '-text.csv')
display(df.head(2))
display(text_df.head(2))

,original,claude_v1_refine
0,<activity_label>Removing ice from car</activit...,<activity_label>הסרת קרח מהרכב</activity_label...
1,<activity_label>Baking cookies</activity_label...,<activity_label>אפיית עוגיות</activity_label>\...


,original
0,<activity_label>Removing ice from car</activit...
1,<activity_label>Baking cookies</activity_label...


In [8]:
!pip install IProgress


In [9]:
exp_name = 'claude_v1_refine'
hebrew_datasets, text_output = claude_translation(
    bedrock_client,
    hellaswag_dataset,
    hallaswag_prompts.HELLASWAG_INSTRUCT_V1_CLAUDE_REFINE,
    hallaswag_prompts.HELLASWAG_FEW_SHOTS,
    hallaswag_prompts.HELLASWAG_FORMAT,
    hellaswag.hellaswag_sample_to_dict,
    hellaswag.hellaswag_dict_to_sample,
)

Translating hellaswag_train...


  0%|          | 0/50 [00:00<?, ?it/s]


KeyboardInterrupt



In [9]:
df = add_dataset_to_csv(hellaswag_file_name, exp_name, hebrew_datasets['hellaswag_train'],  hellaswag.hellaswag_sample_to_dict)
df.head(2)

,original,claude_v1_refine
0,<activity_label>Removing ice from car</activit...,<activity_label>הסרת קרח מהרכב</activity_label...
1,<activity_label>Baking cookies</activity_label...,<activity_label>אפיית עוגיות</activity_label>\...


In [10]:
exp_name = 'gemini_v1_few_options'

hebrew_datasets = gemini_translation(
    google_client,
    hellaswag_dataset,
    hallaswag_prompts.HELLASWAG_INSTRUCT_V1_GEMINI_MULTI,
    hallaswag_prompts.HELLASWAG_FEW_SHOTS,
    hellaswag.hellaswag_sample_to_dict,
    hellaswag.hellaswag_dict_to_sample,
    if_pro=False,
    think_bud=20_000
)

Translating hellaswag_train...


  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
df = add_dataset_to_csv(hellaswag_file_name, exp_name, hebrew_datasets[0]['hellaswag_train'],  hellaswag.hellaswag_sample_to_dict)
df

,original,claude_v1_refine,gemini_v1_few_options
0,<activity_label>Removing ice from car</activit...,<activity_label>הסרת קרח מהרכב</activity_label...,<activity_label>הסרת קרח מהרכב</activity_label...
1,<activity_label>Baking cookies</activity_label...,<activity_label>אפיית עוגיות</activity_label>\...,<activity_label>אפיית עוגיות</activity_label>\...
2,<activity_label>Baking cookies</activity_label...,<activity_label>אפיית עוגיות</activity_label>\...,<activity_label>אפיית עוגיות</activity_label>\...
3,<activity_label>Baking cookies</activity_label...,<activity_label>אפיית עוגיות</activity_label>\...,<activity_label>אפיית עוגיות</activity_label>\...
4,<activity_label>Getting a haircut</activity_la...,<activity_label>קבלת תספורת</activity_label>\n...,<activity_label>הסתפרות</activity_label>\n<ctx...
5,<activity_label>Getting a haircut</activity_la...,<activity_label>קבלת תספורת</activity_label>\n...,<activity_label>תספורת</activity_label>\n<ctx_...
6,<activity_label>Getting a haircut</activity_la...,<activity_label>קבלת תספורת</activity_label>\n...,<activity_label>תספורת</activity_label>\n<ctx_...
7,<activity_label>Getting a haircut</activity_la...,<activity_label>קבלת תספורת</activity_label>\n...,<activity_label>הסתפרות</activity_label>\n<ctx...
8,<activity_label>Getting a haircut</activity_la...,<activity_label>קבלת תספורת</activity_label>\n...,<activity_label>תספורת</activity_label>\n<ctx_...
9,<activity_label>Getting a haircut</activity_la...,<activity_label>קבלת תספורת</activity_label>\n...,<activity_label>תספורת</activity_label>\n<ctx_...
